In [ ]:
import pandas as pd
df1 = pd.read_csv("onsen_info.csv")

def filter_df(df, lat, long):
    threshold = 1
    while(1):
        print(df["東経(゜)"])
        if len(df[(df["東経(゜)"] + 0.01*df["東経(′)"] + 0.0001*df["東経(″)"] - long)**2 + (df["北緯(゜)"] + 0.01*df["北緯(′)"] + 0.0001*df["北緯(″)"] - lat)**2 <= threshold]) == 1:
            break
        else:
            threshold = threshold - 0.1
    return df[(df["東経(゜)"] + 0.01*df["東経(′)"] + 0.0001*df["東経(″)"] - long)**2 + (df["北緯(゜)"] + 0.01*df["北緯(′)"] + 0.0001*df["北緯(″)"] - lat)**2 <= threshold+0.1]

df_test = pd.read_csv("./suisitucsv/愛知.csv")
columns = []
for item in df1.columns:
    columns.append(item)
for item in df_test.columns:
    columns.append(item)
df = pd.DataFrame(columns=columns)

for i, row in df1.query("latitude!='不明'").iterrows():
    print(row)
    pref = row.prefecture
    city = row.city
    name = row.name
    lat = float(row.latitude)
    long = float(row.longitude)
    if pref[-1] != "道":
        pref = pref[:-1]
        df2 = pd.read_csv("./suisitucsv/"+pref+".csv")
    else:
        df2 = pd.concat([pd.read_csv("./suisitucsv/"+pref+"1.csv"), pd.read_csv("./suisitucsv/"+pref+"2.csv")])
    df2 = df2.dropna(subset=["東経(゜)", "東経(′)", "東経(″)", "北緯(゜)", "北緯(′)", "北緯(″)"])

    min_dis = 10000
    nearest_data = pd.Series()
    for index, data in df2[df2["利用目的"].str.contains("温泉")].iterrows():
        try:
            cur_dis = (data["東経(゜)"] + 0.01 * data["東経(′)"] + 0.0001 * data["東経(″)"] - long) ** 2 + (data["北緯(゜)"] + 0.01 * data["北緯(′)"] + 0.0001 * data["北緯(″)"] - lat) ** 2
            if cur_dis < min_dis:
                min_dis = cur_dis
                nearest_data = data
        except TypeError:
            pass
    data = pd.concat([row, nearest_data.fillna(0)])
    df = df.append(data, ignore_index=True)
df.to_csv("merged_onsen_info.csv", index=False)


            
                    



In [ ]:
df3 = pd.read_csv("merged_onsen_info.csv")
df3